In [16]:
import os
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# %%
df = pd.read_csv("../../data/processed/SPLV_clean.csv", parse_dates=["Date"])
test_df = df[df["Date"] >= "2025-03-01"].copy()
X_test = test_df.drop(columns=["target", "Close", "Open", "High", "Low", "Date"])
y_test = test_df["target"]

model_names = ["DecisionTree", "RandomForest", "AdaBoost", "GradientBoosting", "LightGBM", "XGBoost"]
models = {}
for name in model_names:
    path = f"../../models/SPLV_{name}.pkl"
    if os.path.exists(path):
        models[name] = joblib.load(path)

In [18]:
threshold = 0.55
initial_capital = 1000
investment_fraction = 0.25
investment_per_trade = initial_capital * investment_fraction

results = []
for model_name, model in models.items():
    if not hasattr(model, "predict_proba"):
        continue
    y_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_proba > threshold).astype(int)
    temp_df = test_df.copy()
    temp_df["pred"] = y_pred
    temp_df["next_close"] = temp_df["Close"].shift(-1)
    temp_df["return_pct"] = (temp_df["next_close"] - temp_df["Close"]) / temp_df["Close"]
    trade_df = temp_df[temp_df["pred"] == 1].copy()
    trade_df["profit"] = investment_per_trade * trade_df["return_pct"]
    total_profit = trade_df["profit"].sum()
    final_capital = initial_capital + total_profit
    num_trades = len(trade_df)
    win_trades = (trade_df["profit"] > 0).sum()
    loss_trades = (trade_df["profit"] < 0).sum()
    results.append({
        "Model": model_name,
        "Trades": num_trades,
        "Wins": win_trades,
        "Losses": loss_trades,
        "Profit ($)": round(total_profit, 2),
        "Final Capital ($)": round(final_capital, 2)
    })

df_results = pd.DataFrame(results).sort_values(by="Final Capital ($)", ascending=False)
print("💹 Resultados de la simulación con todos los modelos:")
print(df_results)

💹 Resultados de la simulación con todos los modelos:
              Model  Trades  Wins  Losses  Profit ($)  Final Capital ($)
1      RandomForest       6     4       2       15.75            1015.75
5           XGBoost       4     3       1       14.78            1014.78
3  GradientBoosting       2     2       0        3.27            1003.27
2          AdaBoost       0     0       0        0.00            1000.00
4          LightGBM      14     6       8      -10.12             989.88
0      DecisionTree      11     6       4      -12.45             987.55


In [19]:
thresholds = np.arange(0.35, 0.61, 0.05)
for model_key in ["XGBoost", "GradientBoosting", "RandomForest"]:
    model = models[model_key]
    y_proba = model.predict_proba(X_test)[:, 1]
    results = []
    for thresh in thresholds:
        y_pred = (y_proba > thresh).astype(int)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        acc = accuracy_score(y_test, y_pred)
        temp_df = test_df.copy()
        temp_df["pred"] = y_pred
        temp_df["next_close"] = temp_df["Close"].shift(-1)
        temp_df["return_pct"] = (temp_df["next_close"] - temp_df["Close"]) / temp_df["Close"]
        trade_df = temp_df[temp_df["pred"] == 1].copy()
        trade_df["profit"] = investment_per_trade * trade_df["return_pct"]
        total_profit = trade_df["profit"].sum()
        num_trades = len(trade_df)
        wins = (trade_df["profit"] > 0).sum()
        losses = (trade_df["profit"] < 0).sum()
        results.append({
            "Threshold": round(thresh, 2),
            "Accuracy": round(acc, 4),
            "Precision": round(precision, 4),
            "Recall": round(recall, 4),
            "F1-score": round(f1, 4),
            "Trades": num_trades,
            "Wins": wins,
            "Losses": losses,
            "Profit ($)": round(total_profit, 2),
            "Final Capital": round(initial_capital + total_profit, 2)
        })
    df_thresh_results = pd.DataFrame(results).sort_values(by="Profit ($)", ascending=False)
    print(f"\n📈 Comparación de thresholds en {model_key}:")
    print(df_thresh_results.to_string(index=False))


📈 Comparación de thresholds en XGBoost:
 Threshold  Accuracy  Precision  Recall  F1-score  Trades  Wins  Losses  Profit ($)  Final Capital
      0.55    0.6786     0.7500  0.2727    0.4000       4     3       1       14.78        1014.78
      0.60    0.6786     0.7500  0.2727    0.4000       4     3       1       14.78        1014.78
      0.50    0.6429     0.5714  0.3636    0.4444       7     4       3        8.57        1008.57
      0.40    0.5357     0.4444  0.7273    0.5517      18     8       9       -3.68         996.32
      0.45    0.5714     0.4615  0.5455    0.5000      13     6       7       -5.32         994.68
      0.35    0.4643     0.4000  0.7273    0.5161      20     8      11       -7.00         993.00

📈 Comparación de thresholds en GradientBoosting:
 Threshold  Accuracy  Precision  Recall  F1-score  Trades  Wins  Losses  Profit ($)  Final Capital
      0.50    0.6786     1.0000  0.1818    0.3077       2     2       0        3.27        1003.27
      0.55    0.67